# Initial Data Preparation of Telco Churn Dataset

## Imports and Useful Paths

In [1]:
# Helper libraries
import warnings

# Scientific and visual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Various settings
warnings.filterwarnings("ignore")
np.set_printoptions(precision=4)
sns.set_theme()
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.precision", 4)
pd.set_option("display.max_columns", None)

## Getting the Data

In [2]:
from churn_detection.paths import EXTERNAL_DATA_DIR

In [3]:
file_name = "customer_churn.feather"
churn = pd.read_feather(EXTERNAL_DATA_DIR / file_name)

In [4]:
df = churn.copy()

## Data Cleaning

In [5]:
### Target Encoding

In [6]:
df.Churn = df.Churn.map({"Yes":1, "No":0})

In [7]:
### Variable Name Recoding

In [8]:
df.columns = df.columns.str.lower()

In [ ]:
### Irrelevant Data Removal

In [10]:
df.drop(columns="customerid", inplace=True)

In [11]:
### Data Type Correction

In [12]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors="coerce")

In [13]:
df.totalcharges.isna().sum()

np.int64(11)

In [14]:
### Missing Data Correction

In [15]:
missing_charges = [i for i, val in enumerate(df.totalcharges.isna()) if val]
df.loc[missing_charges,:]

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
488,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,0
753,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,0
936,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,0
1082,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,0
1340,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,0
3331,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,0
3826,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,0
4380,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,0
5218,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,0
6670,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,0


In [16]:
# All these customers has tenure=0, thus we suppose they are new customers.
# We can naturally set the NA values to zero:
df.totalcharges = df.totalcharges.fillna(0)

In [17]:
### Inconsistencies in Values

In [ ]:
for col in df.select_dtypes("object").columns:
    df[col] = df[col].str.lower().str.replace(" ", "_")

In [19]:
df.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,female,0,yes,no,1,no,no_phone_service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0
2,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,1
3,male,0,no,no,45,no,no_phone_service,dsl,yes,no,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,0
4,female,0,no,no,2,yes,no,fiber_optic,no,no,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,1


## ML Related Tasks

In [20]:
### Data Splitting: Train, Validation, Test

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
train_full, test = train_test_split(
    df, 
    test_size=0.2, 
    random_state=1
)

In [23]:
train, validation = train_test_split(
    train_full, 
    test_size=0.33, 
    random_state=11
)

In [24]:
train.shape

(3774, 20)

In [25]:
validation.shape

(1860, 20)

In [26]:
test.shape

(1409, 20)

In [27]:
from churn_detection.paths import TRANSFORMED_DATA_DIR

In [28]:
train.to_feather(TRANSFORMED_DATA_DIR / "train.feather")
validation.to_feather(TRANSFORMED_DATA_DIR / "validation.feather")
test.to_feather(TRANSFORMED_DATA_DIR / "test.feather")